# Metropolis Hasting Markov Chain Monte Carlo

- Author: Rudransh Jaiswal

### Algorithm:
Initialize m={m1} (design variable)
For no. of interation n_iters do:
1. Generate random numbers $u1,u2,u3$ ~ U(0,1)
2. - If $u2>0.5$: $m_{new}\; = m+\sigma_m*u1$
 - Else: $m_{new}\; = m-\sigma_m*u1$
3. Calculate P = $ \mathcal{N}(0,\sigma_m ^{2}) | x=S ;\quad S=||Y_{new}-Y||^2$
4. calculate $A=$min$(1,\frac{P_{new}}{P_{old}})$
5. - If A>u3 : accept m = $m_{new}$ 
   - Else m = $m_{old}$ 



In [1]:
import numpy as np
import math
import pandas as pd
pi=np.pi
e=np.e

In [6]:
#MHMCMC Monte Carlo Simulations

#M is array containing initial guess/value of design variable e.g M=[0.2]
#X is independent variable
# Y is dependent on X and m
# F(X,m) gives predicted value of Y at all values of X given some m

def Prob(s,sig): return (pow(2*pi*sig*sig,-0.5))*pow(e,-(0.5*s)/(sig*sig))

def mhmc(X,Y,F,M,sig,n_iters):
    print('## MONTE CARLO SIMULATIONS ##')
    Ypreds=[F(X,M[0])]
    S=[np.linalg.norm(Ypreds[0]-Y)**2]
    P=[Prob(S[0],sig)]
    print('\n----iteration:',0,'----------------')
    print('m:',M[0])    
    cols=['X','Y_pred','Y','Y_pred-Y','Si']
    df=pd.DataFrame([X,Ypreds[0],Y,Ypreds[0]-Y,np.array(Ypreds[0]-Y)*np.array(Ypreds[0]-Y)])
    df=df.T
    df.columns=cols
    print(df)
    print('\nS:',S[0])
    print('P:',P[0])
    i=0
    while n_iters>len(M):
        u1,u2,u3=np.random.uniform(0,1,3)
        print('\n----iteration:',len(M),'----------------')
        print('u1:',u1,'u2:',u2,'u3:',u3)
        m=M[-1]
        if u2>0.5: 
            print('u2>0.5 so m=m+sig*u1=',m,'+',sig,'*',u1)
            m+=sig*u1
        else: 
            print('u2<0.5 so m=m-sig*u1=',m,'-',sig,'*',u1)
            m-=sig*u1
            
        Y_old=Ypreds[-1]
        Y_new=F(X,m)  
        S_old=S[-1]
        S_new=np.linalg.norm(Y_new-Y)**2
        P_old=P[-1]
        P_new=Prob(S_new,sig)
        A=min(1,P_new/P_old)
        
        print('m_new:',m)
        cols=['X','Ynew','Y','Ynew-Y','Si']
        df=pd.DataFrame([X,Y_new,Y,Y_new-Y,np.array(Y_new-Y)*np.array(Y_new-Y)])
        df=df.T
        df.columns=cols
        print(df)
        
        print('\nS_new:',S_new)
        print('P_new',P_new)
        print('P_old',P_old)
        print('ratio:P_new/P_old',P_new/P_old)
        print('A=min(1,ratio)=',A)
        
        if A>u3:
            print('A>u3')
            print('m = m_new=',m)
            M.append(m)
            Ypreds.append(Y_new)
            S.append(S_new)
            P.append(P_new)
            
        else:
            print('A<u3')
            print('m = m_old=',M[-1])
            print('*****redo*****')
        
        i+=1
            
    print('\nFinal Table:')
    ppdf=np.array(P)/sum(P)
    MP=np.array(M)*np.array(P)
    m_mean=sum(MP)/sum(P)
    MM_P=pow(np.array(M)-m_mean,2)*np.array(P)
    m_sig2=sum(MM_P)/sum(P)
    
    df = pd.DataFrame([np.arange(0,n_iters),M,P,MP,MM_P,ppdf])
    df=df.T
    df.columns=['iter','m','P','M*P','(m-m_)^2*P','ppdf']
    print(df)
    
    print('sum(P):',sum(P))
    print('sum(M*P):',sum(MP))
    print('sum(MM_P)',sum(MM_P))
    print('m_mean= sum(M*P)/sum(P)',m_mean)
    print('m_sig2= sum(MM_P)/sum(P)',m_sig2)
    print('m_sig',np.sqrt(m_sig2))
    print('m_map',M[np.argmax(ppdf)])
    
    

Given $\hat Y = a*e^{mx}\quad$a is known constant, m is design variable
- given the datapoints X,Y estimate m using MHMC Monte Carlo


In [7]:
def F1(x,m): return 70.24*pow(e,-m*x)
X=np.array([25,50,75,125])/1000
Y=np.array([90.9,84.47,78.13,65.35])-30
M1=[4.6]
sig1=0.4
mhmc(X,Y,F1,M1,sig1,5)


## MONTE CARLO SIMULATIONS ##

----iteration: 0 ----------------
m: 4.6
       X     Y_pred      Y  Y_pred-Y         Si
0  0.025  62.609558  60.90  1.709558   2.922588
1  0.050  55.808040  54.47  1.338040   1.790352
2  0.075  49.745398  48.13  1.615398   2.609509
3  0.125  39.524390  35.35  4.174390  17.425532

S: 24.747981303098175
P: 2.5800371247945513e-34

----iteration: 1 ----------------
u1: 0.7198169454006916 u2: 0.32043197993872385 u3: 0.5599207543000363
u2<0.5 so m=m-sig*u1= 4.6 - 0.4 * 0.7198169454006916
m_new: 4.312073221839723
       X       Ynew      Y    Ynew-Y         Si
0  0.025  63.061858  60.90  2.161858   4.673630
1  0.050  56.617283  54.47  2.147283   4.610823
2  0.075  50.831308  48.13  2.701308   7.297063
3  0.125  40.972815  35.35  5.622815  31.616048

S_new: 48.19756474953585
P_new 3.859657673711616e-66
P_old 2.5800371247945513e-34
ratio:P_new/P_old 1.495969820209064e-32
A=min(1,ratio)= 1.495969820209064e-32
A<u3
m = m_old= 4.6
*****redo*****

----iteration: 1 --